## Argo File Loader

In [6]:
# try with newest snapshot
# try with ftp that gmaze used in issue, probably going to be really slow

In [4]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import argopy
import scipy.ndimage as filter
import scipy
import matplotlib
import gsw
import traceback

import argopy
from argopy import DataFetcher as ArgoDataFetcher

argo_loader = ArgoDataFetcher(
    src="gdac", ftp="/swot/SUM05/dbalwada/Argo_sync", progress=True
)

In [5]:
print('argopy: {}, xarray: {}, dask: {}'.format(argopy.__version__, xr.__version__, dask.__version__))

argopy: 1.0.0, xarray: 2024.2.0, dask: 2024.12.1


In [7]:
import os
os.chdir('/home.ufs/amf2288/argo-intern/funcs')
import density_funcs as df
import EV_funcs as ef
import filt_funcs as ff
import plot_funcs as pf
import processing_funcs as prf
#import argopy_investigation as ai

In [21]:
reload(df)
reload(ef)
reload(ff)
reload(pf)
#reload(mf)
reload(prf)

<module 'processing_funcs' from '/home/amf2288/argo-intern/funcs/processing_funcs.py'>

## testing parallel loading with dask client

In [ ]:
from dask.distributed import Client

In [ ]:
client= Client(processes=True)

In [ ]:
print(client)

In [ ]:
box = [-27.5, -22.5, -5, 5, 0, 2001]

In [ ]:
%%time
with argopy.set_options(parallel=client):
    f = DataFetcher(src='gdac', ftp='/swot/SUM05/dbalwada/Argo_sync', progress=True).region(box)
    print('%i chunks to process' % len(f.uri))
    print(f)
    ds = f.load().data#.to_xarray()

Okay the process is running on multiple GPUs, in this case 10-12 machines. However, obviously there are a ton of error messages---I'm not really sure what's going wrong. The box I've started with is really small, so I don't think there should be a memory error. Not sure where to go from here.

## Load by box (coordinate: N_PROF)

In [5]:
#box=[lon_min,lon_max,lat_min,lat_max,depth_min,depth_max]
box_atl = [-25,-20,-70,70,0,2001]
box_pac = [-180,-175,-70,70,0,2001]
box_watl = [-60,-55,10,45,0,2001]
box_wpac = [150,155,-5,50,0,2001]
box_wind = [60,65,-65,25,0,2001]

In [6]:
NW = [-180,0,0,90,0,2001]
NE = [0,180,-90,0,0,2001]
SW = [-180,0,-90,0,0,2001]
SE = [0,-180,-90,0,0,2001]

In [7]:
box1 = [-180,-90,  0,90,0,2001]
box2 = [- 90,  0,  0,90,0,2001]
box3 = [   0, 90,  0,90,0,2001]
box4 = [  90,180,  0,90,0,2001]
box5 = [-180,-90,-90, 0,0,2001]
box6 = [- 90,  0,-90, 0,0,2001]
box7 = [   0, 90,-90, 0,0,2001]
box8 = [  90,180,-90, 0,0,2001]

In [8]:
#pf.plot_box([NW,NE,SW,SE])

In [9]:
ds = prf.get_box([-35,-30,-65,-60,0,2000],2)

loading points complete
to xarray complete
point to profile complete
interpolation complete
adding spice complete


IndexError: index -1 is out of bounds for axis 0 with size 0

In [12]:
box = [-35,-30,-65,-60,0,2000]
interp_step = 2

ds = argo_loader.region(box)
print("loading points complete")

ds = ds.to_xarray()
print("to xarray complete")

ds = ds.argo.teos10(["CT", "SA", "SIG0"])
ds = ds.argo.point2profile()
print("point to profile complete")

ds_interp = prf.get_ds_interp(ds, box[4], box[5], interp_step)
print("interpolation complete")

ds_interp["SPICE"] = gsw.spiciness0(ds_interp.SA, ds_interp.CT).rename("SPICE")
print("adding spice complete")

ds_interp = prf.get_MLD(ds_interp)
ds_interp = prf.add_times(ds_interp)
print("adding MLD complete")

if 'raw_attrs' in ds_interp.attrs:
    del ds_interp.attrs['raw_attrs']

loading points complete


100%|██████████| 56/56 [02:47<00:00,  3.00s/it]


to xarray complete
point to profile complete
interpolation complete
adding spice complete


IndexError: index -1 is out of bounds for axis 0 with size 0

In [22]:
ds_interp = prf.get_ds_interp(ds, 0,2000,2)

Using New Function!!


In [28]:
# Compare the profiles before and after interpolation
original_profiles = ds.coords['N_PROF']
interpolated_profiles = ds_interp.coords['N_PROF']

dropped_profiles = set(original_profiles.values) - set(interpolated_profiles.values)
print(f"Dropped profiles: {dropped_profiles}")


Dropped profiles: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47, 48, 50, 53, 57, 60, 61, 65, 69, 70, 71, 72, 73, 74, 76, 81, 83, 84, 85, 86, 88, 90, 91, 93, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 206, 208, 210, 212, 214, 216, 218, 219, 220, 221, 223, 225, 227, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 245, 246, 247, 248, 255, 258, 260, 262, 263, 266, 273, 276, 287, 294, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 316, 320, 321, 322, 323,

In [45]:
ds.sel(N_PROF=23).PRES

<xarray.DataArray 'PRES' (N_LEVELS: 998)> Size: 4kB
array([  16.9,   18. ,   20. ,   22. ,   24. ,   26. ,   28. ,   30. ,
         32. ,   34. ,   36. ,   38. ,   40. ,   42. ,   44. ,   46. ,
         48. ,   50. ,   52. ,   54. ,   56. ,   58. ,   60. ,   62. ,
         64. ,   66. ,   68. ,   70. ,   72. ,   74. ,   76.1,   78. ,
         80. ,   82. ,   84. ,   86. ,   88. ,   90. ,   92. ,   94. ,
         96. ,   98. ,  100. ,  102. ,  104. ,  106. ,  108. ,  110. ,
        112. ,  114. ,  116. ,  118. ,  120. ,  122. ,  124. ,  126. ,
        128. ,  130. ,  132. ,  134. ,  136. ,  138. ,  140. ,  142. ,
        144. ,  146. ,  148. ,  150. ,  152. ,  154. ,  156. ,  158. ,
        160. ,  162. ,  164. ,  166. ,  168. ,  170. ,  172. ,  174. ,
        176. ,  178. ,  180. ,  182. ,  184. ,  186. ,  188. ,  190. ,
        192. ,  194. ,  196. ,  198. ,  200. ,  202. ,  204. ,  206. ,
        208. ,  210. ,  212. ,  214. ,  216. ,  218. ,  220. ,  222. ,
        224. ,  226. ,  228. ,  230. ,  232. ,  234. ,  236. ,  238. ,
        240. ,  242. ,  244. ,  246. ,  248. ,  250. ,  252. ,  254. ,
        256. ,  258. ,  260. ,  262. ,  264. ,  266. ,  268. ,  270. ,
        272. ,  274. ,  276. ,  278. ,  280. ,  282. ,  284. ,  286. ,
        288. ,  290. ,  292. ,  294. ,  296. ,  298. ,  300. ,  302. ,
        304. ,  306. ,  308. ,  310. ,  312. ,  314. ,  316. ,  318. ,
        320. ,  322. ,  324. ,  326. ,  328. ,  330. ,  332. ,  334. ,
...
       1696. , 1698. , 1700. , 1702. , 1704. , 1706. , 1708. , 1710. ,
       1712. , 1714. , 1716. , 1718. , 1720. , 1722. , 1724. , 1726. ,
       1728. , 1730. , 1732. , 1734. , 1736. , 1738. , 1740. , 1742. ,
       1744. , 1746. , 1748. , 1750. , 1752. , 1754. , 1756. , 1758. ,
       1760. , 1762. , 1764. , 1766. , 1768. , 1770. , 1772. , 1774. ,
       1776. , 1778. , 1780. , 1782. , 1784. , 1786. , 1788. , 1790. ,
       1792. , 1794. , 1796. , 1798. , 1800. , 1802. , 1804. , 1806. ,
       1808. , 1810. , 1812. , 1814. , 1816. , 1818. , 1820. , 1822. ,
       1824. , 1826. , 1828. , 1830. , 1832. , 1834. , 1836. , 1838. ,
       1840. , 1842. , 1844. , 1846. , 1848. , 1850. , 1852. , 1854. ,
       1856. , 1858. , 1860. , 1862. , 1864. , 1866. , 1868. , 1870. ,
       1872. , 1874. , 1876. , 1878. , 1880. , 1882. , 1884. , 1886. ,
       1888. , 1890. , 1892. , 1893.9, 1896. , 1898. , 1900. , 1902. ,
       1904. , 1906. , 1908. , 1910. , 1912. , 1914. , 1916. , 1918. ,
       1920. , 1922. , 1924. , 1926. , 1928. , 1930. , 1931.9, 1934. ,
       1936. , 1938. , 1940. , 1942. , 1944. , 1946. , 1948. , 1950. ,
       1952. , 1954. , 1956. , 1958. , 1960. , 1962. , 1964. , 1966. ,
       1968. , 1970. , 1972. , 1974.1, 1976. , 1978. , 1980. , 1982. ,
       1984. , 1986. , 1988. , 1990. , 1992. , 1994. , 1996.1, 1997.3,
          nan,    nan,    nan,    nan,    nan,    nan], dtype=float32)
Coordinates:
    N_PROF     int64 8B 23
  * N_LEVELS   (N_LEVELS) int64 8kB 0 1 2 3 4 5 6 ... 992 993 994 995 996 997
    TIME       datetime64[ns] 8B 2022-05-14T13:38:34
    LATITUDE   float64 8B -64.33
    LONGITUDE  float64 8B -34.84
Attributes:
    long_name:       Sea water pressure, equals 0 at sea-level
    standard_name:   sea_water_pressure
    units:           decibar
    valid_min:       0.0
    valid_max:       12000.0
    C_format:        %7.1f
    FORTRAN_format:  F7.1
    resolution:      1.0
    axis:            Z
    casted:          1

In [44]:
ds.sel(N_PROF=22).PRES

<xarray.DataArray 'PRES' (N_LEVELS: 998)> Size: 4kB
array([  18. ,   20. ,   22. ,   24. ,   26. ,   28. ,   30. ,   32. ,
         34. ,   36. ,   38. ,   40. ,   42. ,   44. ,   46. ,   48. ,
         50. ,   52. ,   54. ,   56. ,   58. ,   60. ,   62. ,   64. ,
         66. ,   68. ,   70. ,   72. ,   74. ,   76. ,   78. ,   80. ,
         82. ,   84. ,   86. ,   88. ,   90. ,   92.1,   94. ,   96.1,
         98. ,  100. ,  102. ,  104. ,  106. ,  108. ,  110. ,  112. ,
        114. ,  116. ,  118. ,  120. ,  122. ,  124. ,  126. ,  128. ,
        130. ,  132. ,  134. ,  136. ,  138. ,  140. ,  142. ,  144. ,
        146. ,  148. ,  150. ,  152. ,  154. ,  156. ,  158. ,  160. ,
        162. ,  164. ,  166. ,  168. ,  170. ,  172. ,  174. ,  176. ,
        178. ,  180. ,  182. ,  184. ,  186. ,  188. ,  190. ,  192. ,
        194. ,  196. ,  198. ,  200. ,  202. ,  204. ,  206. ,  208. ,
        210. ,  212. ,  214. ,  216. ,  218. ,  220. ,  222. ,  224. ,
        226. ,  228. ,  230. ,  232. ,  234. ,  236. ,  238. ,  240. ,
        242. ,  244. ,  246. ,  248. ,  250. ,  252. ,  254. ,  256. ,
        258. ,  260. ,  262. ,  264. ,  266. ,  268. ,  270. ,  272. ,
        274. ,  276. ,  278. ,  280. ,  282. ,  284. ,  286. ,  288. ,
        290. ,  292. ,  294. ,  296. ,  298. ,  300. ,  302. ,  304. ,
        306. ,  308. ,  310. ,  312. ,  314. ,  316. ,  318. ,  320. ,
        322. ,  324. ,  326. ,  328. ,  330. ,  332. ,  334. ,  336. ,
...
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan], dtype=float32)
Coordinates:
    N_PROF     int64 8B 22
  * N_LEVELS   (N_LEVELS) int64 8kB 0 1 2 3 4 5 6 ... 992 993 994 995 996 997
    TIME       datetime64[ns] 8B 2022-08-14T00:16:02.000000256
    LATITUDE   float64 8B -62.48
    LONGITUDE  float64 8B -30.05
Attributes:
    long_name:       Sea water pressure, equals 0 at sea-level
    standard_name:   sea_water_pressure
    units:           decibar
    valid_min:       0.0
    valid_max:       12000.0
    C_format:        %7.1f
    FORTRAN_format:  F7.1
    resolution:      1.0
    axis:            Z
    casted:          1

In [ ]:


for n in range(0, len(ds[dim1])):
        SIG0_surface = ds.isel({dim1: n})[variable].isel({dim2: 0})
        SIG0_diff = SIG0_surface + threshold
        MLD_ds = SIG0_surface.where(ds.isel({dim1: n})[variable] < SIG0_diff)
        MLD = MLD_ds.dropna(dim2).isel({dim2: -1})[dim2].values
        MLD_li.append(MLD)

    return ds.assign_coords(MLD=(dim1, MLD_li))

In [ ]:
ds = ds.assign_attrs({"Fetched_uri":''})
ds

In [ ]:
ds.to_netcdf("/swot/SUM05/amf2288/sync-boxes/lon:({},{})_lat:({},{})_ds_z.nc".format(NW[0],NW[1],NW[2],NW[3]))

In [ ]:
box_1x = [-10,0,-10,0,0,2000]   #error
box_1y = [-10,0,-20,-10,0,2000] #done
box_1z = [-10,0,-30,-20,0,2000] #done
box_1a = [-10,0,-40,-30,0,2000] #done
box_1b = [-10,0,-50,-40,0,2000] #done
box_1c = [-10,0,-60,-50,0,2000] #done
box_1d = [-10,0,-70,-60,0,2000] #done
box_long=[-10,-8,-70,-10,0,2000]

In [ ]:
pf.plot_box([box_1x,box_1y,box_1z,box_1a,box_1b,box_1c,box_1d,box_long])

In [ ]:
ds_z=ff.get_box(box_long,4)

In [ ]:
ds['N_PROF'].attrs = {'name':'Fetched Profile Number'}
ds['PRES_INTERPOLATED'].attrs = {'name':'Interpolated Pressure', 'units':'dbar', 'notes':'interpolated to standard pressure grid of 2m'}

ds['N_PROF_NEW'].attrs = {'name':'Sequential Profile Number'}
ds['LATITUDE'].attrs = {'name':'Latitude', 'units':'°N'}
ds['LONGITUDE'].attrs = {'name':'Longitude', 'units':'°E'}
ds['month'].attrs = {'name':'Month'}
ds['year'].attrs = {'name':'Year'}
ds['MLD'].attrs = {'name':'Mixed Layer Depth', 'units':'m'}

ds['CT'].attrs = {'name':'Conservative Temperature', 'units':'°C'}
ds['SA'].attrs = {'name':'Absolute Salinity', 'units':'g kg-1'}
ds['SIG0'].attrs = {'name':'Potential Density', 'units':'kg m-1', 'notes':'referenced to 0 dbar'}
ds['SPICE'].attrs = {'name':'Spiciness', 'units':'kg m-1', 'notes':'referenced to 0 dbar'}                  
ds = ds.assign_attrs({"Fetched_uri":''})

In [ ]:
ds_z

In [ ]:
plt.scatter(ds_z.LONGITUDE, ds_z.LATITUDE)

In [ ]:
ds_z.to_netcdf("202206_boxes/lon:({},{})_lat:({},{})_ds_z.nc".format(box_long[0],box_long[1],box_long[2],box_long[3]))

In [ ]:
erddap_loader=ArgoDataFetcher(src'erddap',parallel=True,progress=True)

In [ ]:
gmaze_loader=ArgoDataFetcher(src='gdac',ftp='https://data-argo.ifremer.fr/',parallel=True,progress=True)

In [ ]:
usa_loader=ArgoDataFetcher(src='gdac',ftp='ftp://usgodae.org/pub/outgoing/argo',parallel=True,progress=True)

In [ ]:
ds=argo_loader.region(box_1x)
print('loading points complete')
ds=ds.to_xarray()
print('to xarray complete')

In [ ]:
ds=ds.argo.teos10(['CT','SA','SIG0'])
ds=ds.argo.point2profile()
print('point to profile complete')

In [ ]:
def test_get_box(box,sample_min):
    '''Takes latitude/longitude/depth data and a sample rate and returns an xarray with CT, SA, SIG0, and SPICE interpolated to a pressure grid of 2m. 
    
    box: lat/lon in the form: box=[lon_min, lon_max, lat_min, lat_max, depth_min, depth_max]
    sample_min: minimum sample rate [m]
    '''
    
    ds=argo_loader.region(box)
    print('loading points complete')
    
    ds=ds.to_xarray()
    print('to xarray complete')
    
    ds=ds.argo.teos10(['CT','SA','SIG0'])
    ds=ds.argo.point2profile()
    print('point to profile complete')
    
    ds_interp=get_ds_interp(ds,0,2000,sample_min)
    print('interpolation complete')
    
    ds_interp['SPICE'] = gsw.spiciness0(ds_interp.SA,ds_interp.CT).rename('SPICE')
    print('adding spice complete')
        
    return ds_interp

In [ ]:
test = test_get_box(box_medi,4)

In [ ]:
ds=argo_loader.region(box_medi)

In [ ]:
ds = ds.data

In [ ]:
ds

In [ ]:
ds.PSAL.dropna('N_POINTS')

In [ ]:
for n in range(0,len(ds.N_PROF)):
    

In [ ]:
'''
methods are also functions, but specific to the class they're applied to
class (animals) --> subclass (dogs) --> object (golden retriever)
open_mfdataset() from xarray, for opening data in multiple netcdfs
1) write loop opening all files yourself, check for PSAL, if not there remove file
2) go into open_mfdataset() and see if there's a loop there to exploit instead

locally install repo (pip install -e??)
uses the code from the local repo instead
then make changes to repo: print statement in part of the code we think is happening
then work on loops, etc.


'''

In [ ]:
def to_xarray(self, errors: str = "ignore"):
        """ Load Argo data and return a :class:`xarray.Dataset`

        Parameters
        ----------
        errors: str, default='ignore'
            Define how to handle errors raised during data URIs fetching:

                - 'ignore' (default): Do not stop processing, simply issue a debug message in logging console
                - 'silent':  Do not stop processing and do not issue log message
                - 'raise': Raise any error encountered

        Returns
        -------
        :class:`xarray.Dataset`
        """
        if (
            len(self.uri) > 50
            and isinstance(self.method, str)
            and self.method == "sequential"
        ):
            warnings.warn(
                "Found more than 50 files to load, this may take a while to process sequentially ! "
                "Consider using another data source (eg: 'erddap') or the 'parallel=True' option to improve processing time."
            )
        elif len(self.uri) == 0:
            raise DataNotFound("No data found for: %s" % self.indexfs.cname)

        # Download data:
        ds = self.fs.open_mfdataset(
            self.uri, #list of all float files?
            method=self.method,
            concat_dim="N_POINTS",
            concat=True,
            preprocess=self._preprocess_multiprof,
            progress=self.progress,
            errors=errors,
            open_dataset_opts={'xr_opts': {'decode_cf': 1, 'use_cftime': 0, 'mask_and_scale': 1}},
        )

        # Data post-processing:
        ds["N_POINTS"] = np.arange(
            0, len(ds["N_POINTS"])
        )  # Re-index to avoid duplicate values
        ds = ds.set_coords("N_POINTS")
        ds = ds.sortby("TIME")

        # Remove netcdf file attributes and replace them with simplified argopy ones:
        ds.attrs = {}
        if self.dataset_id == "phy":
            ds.attrs["DATA_ID"] = "ARGO"
        if self.dataset_id == "bgc":
            ds.attrs["DATA_ID"] = "ARGO-BGC"
        ds.attrs["DOI"] = "http://doi.org/10.17882/42182"
        ds.attrs["Fetched_from"] = self.server
        ds.attrs["Fetched_by"] = getpass.getuser()
        ds.attrs["Fetched_date"] = pd.to_datetime("now", utc=True).strftime("%Y/%m/%d")
        ds.attrs["Fetched_constraints"] = self.cname()
        if len(self.uri) == 1:
            ds.attrs["Fetched_uri"] = self.uri[0]
        else:
            ds.attrs["Fetched_uri"] = ";".join(self.uri)

        return ds

In [ ]:
print(ds_z.SIG0.values.min(), ds_z.SIG0.values.max())

In [ ]:
rho_grid = np.linspace(26.6, 28, 1001)

In [ ]:
ds_rho = df.interpolate2density_prof(ds_z, rho_grid)

In [ ]:
ds_rho

In [ ]:
ds_rho.to_netcdf("202206_boxes/lon:({},{})_lat:({},{})_ds_rho.nc".format(box[0],box[1],box[2],box[3]))

## Load by float (coordinate: distance)

In [ ]:
float_ID = 6901265

In [ ]:
ds_z = ff.get_float(float_ID, 4)

In [ ]:
ds_z = ds_z.assign_attrs({"Fetched_uri":''})

In [ ]:
ds_z

In [ ]:
ds_z.to_netcdf("202206_floats/float_ID:({})_ds_z.nc".format(float_ID))

In [ ]:
print(ds_z.SIG0.values.min(), ds_z.SIG0.values.max())

In [ ]:
rho_grid = np.linspace(26.4, 28, 1001)

In [ ]:
ds_rho = df.interpolate2density_prof(ds_z, rho_grid)

In [ ]:
ds_rho

In [ ]:
ds_rho.to_netcdf("202206_floats/float_ID:({})_ds_rho.nc".format(float_ID))

## Loading Boxes by Degree

In [ ]:
lons = np.arange(-180,181,4)
lats = np.arange(-90,91,4)

boxes=[]

for n in range(0,len(lons)-1):
    for m in range(0,len(lats-1)):
        lon_min = lons[n]
        lon_max = lons[n+1]
        lat_min = lats[n]
        lat_max = lats[n+1]
        box_n = np.array([lon_min, lon_max, lat_min, lat_max])
        boxes.append(box_n)

In [ ]:
n=0
lons[n+1]

In [ ]:
from argopy import IndexFetcher as ArgoIndexFetcher

box_bad=[-15,-14,36.5,37]#, '2000-01-01', '2021-06'] # need to remove pressure poinds because the region function for this index fetcher seems a bit different.

idx = ArgoIndexFetcher(src='gdac',dataset='phy',mode='standard',
                            ftp="/swot/SUM05/dbalwada/202203-ArgoData").region(box_bad).load()
idx.index

In [ ]:
float_IDs = [1900041,1900749,]